# Scraping

In this first step, we will capture the data from the internet. The idea is to create a scraper to gather data regarding house pricing. Now, there are a lot of pages where we could obtain that data. To this specific exercise, we will use the folloging web pages;

 * https://casas.trovit.com.co/arriendo-valle-aburra
 * https://www.espaciourbano.com/listado_arriendos.asp?sZona=1
 * https://inmobiliariasantillana.co/propiedades/?filter=true&tipo_de_negocio=Arriendo
 * https://www.fincaraiz.com.co/arriendo?&searchstring=bello
 * https://www.fincaraiz.com.co/arriendo/casas-y-apartamentos-y-edificios-y-apartaestudios-y-cabanas-y-casas-campestres/antioquia
 * https://www.metrocuadrado.com/apartaestudio-apartamento-casa-edificio-de-apartamentos/arriendo/medellin/?search=form
  
NOTA: fincaraiz.com requieres Selenium. metrocuadrado.com as well. 